# Setup installs & imports

In [ ]:
## install required packages
!pip install swig
!pip install wrds
!pip install pyportfolioopt
## install finrl library
!pip install git+https://github.com/AI4Finance-Foundation/FinRL.git

  Cloning https://github.com/AI4Finance-Foundation/FinRL.git to /tmp/pip-req-build-95b9hvj3
  Running command git clone --filter=blob:none --quiet https://github.com/AI4Finance-Foundation/FinRL.git /tmp/pip-req-build-95b9hvj3
  Resolved https://github.com/AI4Finance-Foundation/FinRL.git to commit 6156c42121c21fa153722c9bbf4deef3f8f79a79
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Cloning https://github.com/AI4Finance-Foundation/ElegantRL.git to /tmp/pip-install-vyxan_77/elegantrl_46cab076fa2c4085822c5dba34eba62b
  Running command git clone --filter=blob:none --quiet https://github.com/AI4Finance-Foundation/ElegantRL.git /tmp/pip-install-vyxan_77/elegantrl_46cab076fa2c4085822c5dba34eba62b
  Resolved https://github.com/AI4Finance-Foundation/ElegantRL.git to commit 307b4a264b256bdb21cc481456183bc99e266be9
  Preparing metadata (setup.py) ... done


In [ ]:
import os
import pandas as pd

from finrl.meta.env_stock_trading.env_stocktrading import StockTradingEnv  ##creates a custom enviorunemnt known as stockstrading env designed only for stock trading
from finrl.agents.stablebaselines3.models import DRLAgent   ###use of reinforcement learning algorithms provided by the Stable-Baselines3 library.
## It’s used to train, evaluate, and deploy deep reinforcement learning models.
from stable_baselines3.common.logger import configure  ###configure function is used to set up logging for training, which helps in tracking the progress
## and performance of the model during training.
from finrl import config_tickers  ##is a list of stock symbols (like “AAPL” for Apple)
from finrl.main import check_and_make_directories
from finrl.config import INDICATORS, TRAINED_MODEL_DIR, RESULTS_DIR

check_and_make_directories([TRAINED_MODEL_DIR])

In [ ]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


# Load the Data from Part 1

In [ ]:
train = pd.read_csv('/content/drive/My Drive/train.csv')
train=train.set_index(train.columns[0])
train.index.names=['']


## Setup Environment

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
stock_dimension=len(train.tic.unique())
state_space=1+2*stock_dimension+len(INDICATORS)*stock_dimension    # state space provides the agent
##with all the necessary information to evaluate its current situation and decide on the next action (e.g., buy, sell, hold a stock).
print(f"Stock Dimension: {stock_dimension}, State Space: {state_space}")

Stock Dimension: 8, State Space: 81


In [ ]:
buy_cost_list = sell_cost_list = [0.001] * stock_dimension   #transaction cost
num_stock_shares = [0] * stock_dimension   ###list keeps track of the number of shares held for each stock.
##meaning no shares are held for any stock at the beginning.
env_kwargs={     ##variable name
    "hmax":100,   #maximum number of shares
    "initial_amount":1000000, # Changed "initalization_amount" to "initial_amount"
    "num_stock_shares":num_stock_shares,
    "buy_cost_pct":buy_cost_list,
    "sell_cost_pct":sell_cost_list,
    "state_space":state_space,
    "stock_dim":stock_dimension,
    "tech_indicator_list":INDICATORS,
    "action_space":stock_dimension,
    "reward_scaling":1e-4
}
e_train_gym = StockTradingEnv(df = train, **env_kwargs)

### Environment for Training

In [ ]:
env_train, _ = e_train_gym.get_sb_env()   ###onverts your custom environment(ENVKAWRDS CONVERTED TO E_TRAIN GYM) into a format compatible with stable-baselines3
##, a popular library for reinforcement learning.
print(type(env_train))

<class 'stable_baselines3.common.vec_env.dummy_vec_env.DummyVecEnv'>


# Train Agent

In [ ]:
agent = DRLAgent(env = env_train)

# Set the corresponding values to 'True' for the algorithms that you want to use
if_using_a2c = True
if_using_ddpg = False
if_using_ppo =  False
if_using_td3 = False
if_using_sac = False

## A2C Model

The code above and below provide a healthy framework to add different agent types for training within the same notebook. For this I'll only be training an a2c model, but may add ddpg, ppo, and td3 in later iterations.

In [ ]:
model_a2c = agent.get_model("a2c")

if if_using_a2c:
  #setting up logger
  tmp_path = RESULTS_DIR + '/a2c'
  new_logger = configure(tmp_path, ["stdout", "csv", "tensorboard"])
  # Set new logger
  model_a2c.set_logger(new_logger)
  # - "stdout": Logs will be printed to the console.
# - "csv": Logs will be saved in CSV format, useful for later analysis.
# - "tensorboard": Logs will be compatible with TensorBoard for visualization.


{'n_steps': 5, 'ent_coef': 0.01, 'learning_rate': 0.0007}
Using cpu device


In [ ]:
trained_a2c=agent.train_model(model=model_a2c,tb_log_name='a2c',total_timesteps=54000) if if_using_a2c else None

---------------------------------------
| time/                 |             |
|    fps                | 155         |
|    iterations         | 100         |
|    time_elapsed       | 3           |
|    total_timesteps    | 500         |
| train/                |             |
|    entropy_loss       | -13         |
|    explained_variance | -1.19e-07   |
|    learning_rate      | 0.0007      |
|    n_updates          | 40896       |
|    policy_loss        | 4.41        |
|    reward             | -0.71258926 |
|    std                | 1.24        |
|    value_loss         | 0.66        |
---------------------------------------
-------------------------------------
| time/                 |           |
|    fps                | 137       |
|    iterations         | 200       |
|    time_elapsed       | 7         |
|    total_timesteps    | 1000      |
| train/                |           |
|    entropy_loss       | -13       |
|    explained_variance | 0         |
|    learning_rate

In [ ]:
trained_a2c.save(TRAINED_MODEL_DIR + "/content/drive/My Drive/financial project/agent_a2c") if if_using_a2c else None


/usr/local/lib/python3.10/dist-packages/stable_baselines3/common/save_util.py:284: UserWarning: Path 'trained_models/content/drive/My Drive/financial project' does not exist. Will create it.
  warnings.warn(f"Path '{path.parent}' does not exist. Will create it.")


# Consider Joining the Newsletter
Stay Notified of New Videos and Code

https://witty-motivator-1414.ck.page/acb393f729